In [1]:
import spacy

# Load SpaCy's pre-trained model
nlp = spacy.load('en_core_web_sm')

In [50]:
s = "May I have a medium iced coffee?"
s.find('medium')

13

In [51]:
TRAIN_DATA = [
    ("I would like to order a large pizza with extra cheese.", {"entities": [(30, 34, "FOOD"), (24, 28, "SIZE"), (22,22, "QUANTITY")]}),
    ("Can I get a small coke, please?", {"entities": [(18, 22, "FOOD"), (12,16, "SIZE"), (10,10, "QUANTITY")]}),
    ("I want two medium plates of chicken biryani.", {"entities": [(21, 35, "FOOD"), (11,16, "SIZE"), (7, 9, "QUANTITY")]}),
    ("May I have a medium iced coffee?", {"entities": [(20, 30, "FOOD"), (13, 18, "SIZE"), (11,11, "QUANTITY")]}),
    
    ("I'll take a large fries with ketchup.", {"entities": [(13, 18, "FOOD"), (7, 12, "SIZE"), (5, 6, "QUANTITY")]}),
    ("Give me three slices of pepperoni pizza.", {"entities": [(20, 33, "FOOD"), (9, 14, "SIZE"), (5, 6, "QUANTITY")]}),
    ("Can I have a small bottle of water?", {"entities": [(17, 22, "FOOD"), (10, 15, "SIZE"), (2, 3, "QUANTITY")]}),
    ("I'd like to order two large burgers.", {"entities": [(24, 30, "FOOD"), (11, 16, "SIZE"), (7, 8, "QUANTITY")]}),
    ("May I have a regular coke?", {"entities": [(21, 24, "FOOD"), (13, 19, "SIZE"), (11, 12, "QUANTITY")]}),
    ("Give me a small portion of french fries.", {"entities": [(24, 36, "FOOD"), (10, 15, "SIZE"), (5, 6, "QUANTITY")]}),
]

In [52]:
# Get the NER component from the pipeline
ner = nlp.get_pipe("ner")

# Add your custom entity labels to the model
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [53]:
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example

# Disable unnecessary pipeline components
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*unaffected_pipes):
    # Training loop
    n_iter = 10
    batch_size = 16

    for itn in range(n_iter):
        losses = {}
        random.shuffle(TRAIN_DATA)
        batches = minibatch(TRAIN_DATA, size=batch_size)
        for batch in batches:
            examples = []
            for text, annot in batch:
                examples.append(Example.from_dict(nlp.make_doc(text), annot))
            nlp.update(
                examples,
                drop=0.5,  # Dropout rate
                losses=losses,
            )
        print(f"Iteration {itn}: Losses - {losses}")


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I'd like to order two large burgers." with entities "[(24, 30, 'FOOD'), (11, 16, 'SIZE'), (7, 8, 'QUANT...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I would like to order a large pizza with extra che..." with entities "[(30, 34, 'FOOD'), (24, 28, 'SIZE'), (22, 22, 'QUA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the te

Iteration 0: Losses - {'ner': 5.986477489905673}
Iteration 1: Losses - {'ner': 7.39003442512967}
Iteration 2: Losses - {'ner': 5.510321385506636}
Iteration 3: Losses - {'ner': 6.180595066220889}
Iteration 4: Losses - {'ner': 5.621758089729212}
Iteration 5: Losses - {'ner': 5.636817289684302}
Iteration 6: Losses - {'ner': 4.790144825437345}
Iteration 7: Losses - {'ner': 7.01465390467456}
Iteration 8: Losses - {'ner': 5.214767193474472}
Iteration 9: Losses - {'ner': 5.147595077625738}


In [54]:
# Save the trained model
output_dir = 'path_to_output_directory'
nlp.to_disk(output_dir)

In [55]:
# Load the trained model
nlp = spacy.load(output_dir)

# Test the model
test_text = "I would like to order a large pizza with extra cheese"
doc = nlp(test_text)
for ent in doc.ents:
    print(ent.text, ent)

In [56]:
print(doc.ents)

()


In [57]:
s = "Can I get a small coke, please?"

In [58]:
s[16]

'l'